<a href="https://colab.research.google.com/github/mjn6862/Speed_Challenge/blob/master/Matthew_RNN_Speed_Challenge_Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Comma AI Speed Challenge**

  This notebook will contain (hopefully) all of the functions you need to import the data into your model.

  ***Be sure to train with GPU acceleration enabled***

**Import Statements**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from datetime import datetime
import os

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

**Custom Data Generator**

This works (I think) for giving two sequential images to a Keras Functional model as well as the velocity associated with the second image.

At this point, don't worry about how this works. If you need something changed or fixed, just ask. This is the boring part anyways.

In [ ]:
# need to mount drive before using this
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.direct = "./drive/My Drive/commai_dataset/"

    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(np.floor(len(self.list_IDs) / self.batch_size))
        return len(self.list_IDs)
        
    def __getitem__(self, index):
        'Generate one batch of data'
        
        # Find list of IDs
        list_IDs_temp = self.list_IDs[index]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.load(self.direct+"data/data_" + list_IDs_temp +".npy")
        X = X/255
        x1 = X[0:101,:,:,:]
        x2 = X[1:102,:,:,:]
        y = np.load(self.direct+"labels/label_" + list_IDs_temp +".npy")
        y = y[1:]
        y.reshape(101,1)

        return [np.array([x1]),np.array([x2])], y

**Define custom loss function**

This is not well tested, neither is it optimized. You might not even want to use this function.

Keras backend functions are a powerful tool for writing custom loss functions. To define a loss function it just has to accept *y_true* and *y_pred* as arguments and return a float.

To use your new loss function, change the argument in *model.compile()*.

**Define the test-train split and create the Data Generator**

In [ ]:
params = {'dim': (101,110,320),
          'batch_size': 1,
          'n_classes': 1,
          'n_channels': 3,
          'shuffle': False}

train_data = []
train_label = []
valid_data = []
valid_label = []

for i in range(70):
    train_data.append("%03d" %i)
    train_data.append("%03d" %(i+100))

for i in range(70, 100):
    valid_data.append("%03d" %i)
    valid_data.append("%03d" %(i+100))

partition={'train':train_data, 'validation':valid_data}
labels = {'train': train_label,'validation':valid_label}

training_generator = DataGenerator(partition['train'], labels['train'], **params)
validation_generator = DataGenerator(partition['validation'], labels['validation'], **params)

**Define the input layers**

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, ConvLSTM2D, TimeDistributed, LSTM, Dense, Flatten, concatenate

In [ ]:
inputA = Input(shape=(101,110,320,3), name='first_image') # for RNN shape=(time_steps, pixels, pixels, rgb)
inputB = Input(shape=(101,110,320,3), name='second_image')

**Define the model**



In [ ]:
convA = TimeDistributed(Conv2D(24,(3,3),activation='relu',name='convA'))(inputA)
convB = TimeDistributed(Conv2D(24,(3,3),activation='relu',name='convB'))(inputB)

conc = concatenate(inputs=[convA,convB],name='conc')

conv11 = TimeDistributed(Conv2D(32,(3,3),strides=(2,2),activation='relu',name='conv11'))(conc)
block1 = TimeDistributed(Conv2D(32,(3,3),strides=(2,2),activation='relu',name='block1'))(conv11)

conv21 = ConvLSTM2D(64,(3,3),activation='tanh', return_sequences=True, name='conv21')(block1)
block2 = TimeDistributed(Conv2D(64,(3,3),strides=(2,2),activation='relu',name='block2'))(conv21)

conv31 = TimeDistributed(Conv2D(128,(3,3),activation='relu', name='conv31'))(block2)
block3 = TimeDistributed(Conv2D(128,(3,3),strides=(2,2),activation='relu', name='block3'))(conv31)

flat = TimeDistributed(Flatten(name='flat'))(block3)

output = TimeDistributed(Dense(1, name='output'))(flat)

In [ ]:
model = tf.keras.Model(inputs=[inputA,inputB], outputs=output)
#model.load_weights('/content/drive/My Drive/SpeedChallenge/Checkpoints/jrk_match20_2_2.h5', by_name=True)

In [ ]:
model.summary()

**Declare the optimizer and loss function, then compile your *less ridiculous*  model**

In [ ]:
import tensorflow.keras.backend as K

delta = 10
def acc_min(y_true, y_pred):
  return K.mean(K.square(y_true-y_pred)) + delta*K.sum(K.square(y_pred[1:]-y_pred[0:-1]))

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(clipnorm=0.001),
               loss=acc_min,
              metrics=['mse'])

**Prepare the TensorBoard and Callbacks**

In [ ]:
logdir = os.path.join("speed_logs2", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
earlystop_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
# save weights at each epoch
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="./drive/My Drive/SpeedChallenge/Checkpoints/rnn_{epoch}.h5", save_weights_only=True, save_freq='epoch', verbose=0)

In [ ]:
keep = 30 # how many epochs to keep 0.001 as the learning rate
def scheduler(epoch):
  if epoch < keep:
    return 0.0005
  else:
    return 0.0005*tf.math.exp(0.1*(keep-epoch))

schedule_callback = keras.callbacks.LearningRateScheduler(scheduler)

**Train using the fit_generator**

In [ ]:
model.fit_generator(generator=training_generator, 
                    validation_data=validation_generator, 
                    verbose=1,
                    epochs=30,
                    callbacks=[tensorboard_callback,
                               cp_callback,
                               schedule_callback])

In [ ]:
%tensorboard --logdir \speed_logs

**Visualize the predictions**


In [ ]:
output3 = keras.layers.ReLU()(output)
prediction_model = keras.Model(inputs=[inputA,inputB],outputs=output3)
prediction_model.set_weights(model.get_weights())

In [ ]:
#prediction_model.load_weights('/content/drive/My Drive/SpeedChallenge/Checkpoints/rnn_5.h5', by_name=True)

In [ ]:
predictions = model.predict(validation_generator)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# get all validation labels from the generator into one array
val_labels = np.zeros(np.prod(predictions.shape))
for i in range(predictions.shape[0]):
  val_labels[i*101:(i+1)*101] = validation_generator[i][1]

In [ ]:
# sort predictions into size mph bins based on ground truth labels
flat_predictions = np.ndarray.flatten(predictions)
size = 2
sorted_predictions = []
sort_bins = np.arange(0,26+size,size) # max label is 26.05
pred_bins = np.arange(0,26+size,size)
for i in range(len(sort_bins)-1):
  preds = flat_predictions[np.where((sort_bins[i] <= val_labels) & (val_labels < sort_bins[i+1]))]
  sorted_predictions.append(preds)

In [ ]:
# use histograms to see how well each size mph bin was predicted
num_rows = len(sort_bins)//2 # + 1 sometimes
num_cols = 2
num_images = num_rows*num_cols
plt.figure(figsize=(4*2*num_cols, 4*num_rows))
for i in range(num_images-1):
  plt.subplot(num_rows,2*num_cols, 2*i+1)
  plt.hist(sorted_predictions[i], bins = pred_bins)
  plt.xticks(ticks=pred_bins,rotation=-30)
  plt.title('{}-{} mph'.format(sort_bins[i],sort_bins[i+1]))
plt.tight_layout()
plt.show()

In [ ]:
prediction_model.compile(optimizer=keras.optimizers.Adam(),
               loss='mean_squared_error')

In [ ]:
prediction_model.evaluate(validation_generator)